In [1]:
import keras

Using TensorFlow backend.


In [16]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
data=open("../NLG/summary_types.txt", 'r', encoding='utf-8').read()

In [4]:
data

'The above graph shows sales history and sales forecast for product PROD_ID at LOC_ID.\nThe sales history is taken into consideration is between SALES_HISTORY_SDATE and SALES_HISTORY_EDATE. \nAs we can see in the graph it attained its peak of PEAK_VAL which was recorded at PEAK_DATE and recorded lowest sales of MINVAL at MINDATE.\nThe average sales during this period was AVG_SALES.\nThe year wise mean for years START_YEAR to END_YEAR are YEALRY_AVG and the yearly trend for year START_YEAR to END_YEAR are TREND.\nThe Finalized planner forecast predicts the future trend for period PFS_YEAR to PFE_YEAR, it predicts the maximum sales of PMAX_SALES on PMAX_DATE and lowest sales recording PMIN_SALES at PMIN_DATE.\nThe finalized planner forecast predicts an average sales of FP_AVG for this period.\nFor Finalized planner forecast the year wise mean for years FP_START_YEAR to FP_END_YEAR are FPF_YEARLY_AVG and the yearly TREND for year FP_START_YEAR to FP_END_YEAR are FPS_TREND.\nSimilarly, the

In [5]:
chars = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:

n_chars = len(data)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)


Total Characters:  12817
Total Vocab:  49


In [8]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = data[i:i + seq_length]
    seq_out = data[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  12717


In [10]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [14]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [18]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

W0913 16:30:46.301225  1928 deprecation.py:323] From C:\D\Installations\Anaconda\envs\tensorflow_env\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 12717 samples
Epoch 1/20
11776/12717 [==========================>...] - ETA: 5s - loss: 3.4244

KeyboardInterrupt: 

In [ ]:
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')